In [ ]:
from pyrealm.pmodel import (
    SubdailyScaler,
    memory_effect,
    SubdailyPModel,
    PModelEnvironment,
    PModel,
)
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pyrealm.pmodel.optimal_chi import OptimalChiPrentice14
from pyrealm.pmodel.functions import calc_ftemp_arrh, calc_ftemp_kphio,calc_modified_arrhenius_factor
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pyrealm.constants import PModelConst
pmodel_const = PModelConst()
from pyrealm.pmodel import (
    SubdailyScaler,
    memory_effect,
    SubdailyPModel,
    PModelEnvironment,
    PModel,
)
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pyrealm.pmodel.optimal_chi import OptimalChiPrentice14
from pyrealm.pmodel.functions import calc_ftemp_arrh, calc_ftemp_kphio, calc_modified_arrhenius_factor
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pyrealm.constants import PModelConst

In [ ]:
# Get enzyme kinetics parameters
a, b, ha, hd = pmodel_const.kattge_knorr_kinetics
# Calculate entropy as a function of temperature _in °C_
deltaS = a + b * 10

# Number of half-hourly data points
n_pts = 17520  # 365 days * 48 half-hours

# Build a datetime index at 30-minute intervals
start_datetime = pd.to_datetime('2023-01-01')
datetimes = pd.date_range(start=start_datetime, periods=n_pts, freq='30T')

# Convert to a NumPy datetime64[m] array
datetimes_np = datetimes.to_numpy(dtype='datetime64[m]')

# Atmospheric pressure (Pa)
patm = np.full(n_pts, 101325)  # Standard atmospheric pressure

# VPD (vapor pressure deficit, Pa)
vpd = np.full(n_pts, 1000)  # Assume VPD = 500 Pa

# fAPAR (unitless)
fapar = np.full(n_pts, 1)  # Assume full canopy cover

# PPFD (µmol m⁻² s⁻¹)
ppfd = np.full(n_pts, 1200)  # PPFD fixed

# Atmospheric CO₂ concentration (ppm)
co2 = np.full(n_pts, 400)  # One value per half hour

# List of acclimation temperatures
acclim_temps = np.arange(10, 41, 5)

# Create SubdailyScaler and set the acclimation window
fsscaler = SubdailyScaler(datetimes=datetimes_np)
fsscaler.set_window(
    window_center=np.timedelta64(12, 'h'),   # 12:00 noon
    half_width=np.timedelta64(30, 'm'),      # ±0.5 hour
)

# Create a mask for 11:30–12:30 (acclimation window)
times = datetimes.time
start_time = pd.to_datetime('11:30').time()
end_time = pd.to_datetime('12:30').time()
acclim_mask = (times >= start_time) & (times <= end_time)

# Initialize containers for results
gpp_results = []
vcmax_results = []
jmax_results = []
temp_results = []
ac_results = []
aj_results = []
results = []


In [ ]:
# 👇 define correct sandoval
from pyrealm.pmodel.quantum_yield  import QuantumYieldABC
import numpy as np
from warnings import warn

class QuantumYieldSandoval(
    QuantumYieldABC,
    method="sandoval",
    requires=["aridity_index", "mean_growth_temperature"],
    default_reference_kphio=1.0 / 9.0,   
    array_reference_kphio_ok=False
):
    """Overridden Sandoval-style quantum yield using Medlyn-style Arrhenius."""

    __experimental__: bool = True

    def peak_quantum_yield(self, aridity_index: np.ndarray) -> np.ndarray:
        phi0_theo = 1.0 / 9.0
        m = 4.090556
        n = 0.121122
        return phi0_theo / (1 + aridity_index**m)**n

    def _calculate_kphio(self) -> None:
        warn("QuantumYieldSandoval (overridden) is experimental.")
        
        ai = self.env.aridity_index
        mgdd = self.env.mean_growth_temperature
        tcleaf = self.env.tc
        tkleaf = self.env.tc + 273.15
        R = self.env.core_const.k_R

        dS0 = 3468.185
        dS_mgdd = 0.6680158
        Ha = 70885.39

        DeltaS = dS0 * mgdd**(-dS_mgdd)
        Hd = 295.0 * DeltaS

        Top = Hd / (DeltaS - R * np.log(Ha / (Hd - Ha)))

        def medlyn_peaked_arrhenius(tk, Top, Ha, Hd, dS):
            f1 = Hd * np.exp(Ha * (tk - Top) / (tk * R * Top))
            f2 = Hd - Ha * (1 - np.exp(Hd * (tk - Top) / (tk * R * Top)))
            return f1 / f2

        phi0_peak = self.peak_quantum_yield(ai)
        phi0_fT = medlyn_peaked_arrhenius(tkleaf, Top, Ha, Hd, DeltaS)

        self.kphio = phi0_peak * phi0_fT
QuantumYieldSandoval = QuantumYieldSandoval

In [ ]:
from pyrealm.pmodel.optimal_chi import OptimalChiABC
import numpy as np

# define fixed chi
class OptimalChiFixed(OptimalChiABC, method="fixed", is_c4=False, requires=("fixed_chi",)):
    """
    自定义的固定 Chi 计算类。
    包含从 Chi 反推 Xi 的逻辑。
    """
    
    def set_beta(self) -> None:
        """设置默认 Beta 值 (仅为满足 API 需求)。"""
        self.beta = self.pmodel_const.beta_cost_ratio_prentice14

    def estimate_chi(self, xi_values=None) -> None:
        """
        使用固定的 Chi，并反向计算 Xi。
        """
        # 1. 获取固定 Chi 值
        if not hasattr(self.env, "fixed_chi"):
             raise ValueError("PModelEnvironment missing required 'fixed_chi' variable.")
             
        self.chi = getattr(self.env, "fixed_chi")

        # 2. 计算 Ci, mj, mc (这些步骤不变)
        self.ci = self.chi * self.env.ca
        self.mj = (self.ci - self.env.gammastar) / (self.ci + 2 * self.env.gammastar)
        self.mc = (self.ci - self.env.gammastar) / (self.ci + self.env.kmm)
        self.mjoc = self.mj / self.mc
        
        # 3. 【新逻辑】反向计算 Xi
        #    公式: Xi = (Chi * sqrt(VPD)) / (1 - Chi)
        #    注意：如果 Chi 固定为 1.0，这里会发生除以零错误，需要注意
        
        vpd_sqrt = np.sqrt(self.env.vpd)
        
        # 为了防止除以0 (当 chi=1 时)，加一个极小的 epsilon 或者确保 chi < 1
        # 这里假设 chi 不会完全等于 1
        denominator = 1.0 - self.chi
        
        # 计算 Xi
        self.xi = (self.chi * vpd_sqrt) / denominator

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import matplotlib.colors as mcolors

# 假设外部已定义变量：
# n_pts, patm, vpd, co2, fapar, acclim_mask, fsscaler
# memory_effect, calc_kmm, calc_gammastar, OptimalChiPrentice14
# SubdailyPModel, PModel, PModelEnvironment

acclim_temps = np.arange(0, 41, 2.5)
co2 = np.full(n_pts, 420)  # One value per half hour

# ---------- Configuration ----------
light_levels = [250,600, 1200, 1800]      
temps = list(acclim_temps)            

# Colors
norm = mcolors.Normalize(vmin=min(temps), vmax=max(temps))
cmap = cm.get_cmap('coolwarm')

# Store results
results_by_light = {L: [] for L in light_levels}

# ---------- Main loop ----------
for acclim_ppfd in light_levels:
    for acclim_temp in temps:
        # 1) Construct instantaneous forcing
        tc_1d = np.linspace(-10, 60, n_pts).copy()
        patm_use = patm.copy()
        vpd_use  = vpd.copy()
        co2_use  = co2.copy()
        fapar_use = fapar.copy()
        ppfd_use  = np.full(n_pts, 600.0)    
        tc_1d[acclim_mask] = acclim_temp      
        ppfd_use[acclim_mask] = acclim_ppfd    

        # Daily means
        co2_acclim  = fsscaler.get_daily_means(co2_use)
        vpd_acclim  = fsscaler.get_daily_means(vpd_use)
        patm_acclim = fsscaler.get_daily_means(patm_use)

        # Environment (Subdaily)
        env = PModelEnvironment(tc=tc_1d, patm=patm_use, vpd=vpd_use, co2=co2_use)
        env.aridity_index = np.full(n_pts, 0.0)    
        env.mean_growth_temperature = np.full(n_pts, 25)
        
        # Initial Subdaily Model (for parameters)
        subdailyC3 = SubdailyPModel(
            env=env,
            method_kphio='sandoval',
            reference_kphio=1/9,
            method_optchi='prentice14',
            fapar=fapar_use,
            ppfd=ppfd_use, 
            fs_scaler=fsscaler,
            alpha=1/15,
            allow_holdover=True,
        )

        # 2) Acclimation Model (Daily)
        acclim_env = PModelEnvironment(
            tc=acclim_temp,
            patm=patm_acclim,
            vpd=vpd_acclim,
            co2=co2_acclim
        )
        acclim_env.aridity_index = 0
        acclim_env.mean_growth_temperature = np.array([25])
        # acclim_env.kmm = np.full_like(acclim_env.kmm, 256)
        # # acclim_env.fixed_chi = np.full_like(acclim_env.kmm,0.7)
        # acclim_env.gammastar = np.full_like(acclim_env.gammastar,4.33)
        pmodel_acclim = PModel(
            env=acclim_env,
            method_kphio='sandoval',
            reference_kphio=1/9,
            method_optchi='prentice14'
        )
        
        # 修正：这里应该使用 env 而不是未定义的 pmodel_env
        pmodel_instant = PModel(
            env=env, 
            method_kphio='sandoval',
            reference_kphio=1/9,
            method_optchi="prentice14",
        )
        
        pmodel_acclim.estimate_productivity(fapar=1, ppfd=acclim_ppfd)
        
        # Constants
        R = 8.3145 
        ha_jmax25 = 43900 
        ha_vcmax25 = 65330 
        
        # 3) Calculate Acclimated Traits (Removing Temp Effect)
        vcmax25_acclim = pmodel_acclim.vcmax * np.exp((ha_vcmax25/R)*(1/(acclim_temp+273.15)-1/298.15))
        jmax25_acclim = pmodel_acclim.jmax * np.exp((ha_jmax25/R)*(1/(acclim_temp+273.15)-1/298.15))

        # Memory Effect
        xi_real = memory_effect(pmodel_acclim.optchi.xi, alpha=1/15)
        vcmax25_real = memory_effect(vcmax25_acclim, alpha=1/15, allow_holdover=True)
        jmax25_real = memory_effect(jmax25_acclim, alpha=1/15, allow_holdover=True)

        # Fill to Subdaily
        vcmax25_subdaily = fsscaler.fill_daily_to_subdaily(vcmax25_real)
        jmax25_subdaily = fsscaler.fill_daily_to_subdaily(jmax25_real)

        # Adjust to Actual Temperature (Subdaily)
        vcmax_subdaily = vcmax25_subdaily * np.exp((ha_vcmax25/R)*(1/298.15-1/(tc_1d+273.15)))
        jmax_subdaily = jmax25_subdaily * np.exp((ha_jmax25/R)*(1/298.15-1/(tc_1d+273.15)))
        
        # Instantaneous Chi
        xi_subdaily = fsscaler.fill_daily_to_subdaily(xi_real)
        # Calculate optimal chi
        # 修正：这里同样应该使用 env
        subdaily_chi = OptimalChiPrentice14(env=env) 
        subdaily_chi.estimate_chi(xi_values=xi_subdaily)
        
        ci_subdaily = subdaily_chi.ci
        # kmm = calc_kmm(tc_1d, patm.transpose()) # 如果 env 里已经有了，可以直接用 env.kmm
        kmm = env.kmm.flatten() # 建议直接使用 PyRealm 计算好的 kmm
        
        fkphio_c3 = subdailyC3.kphio.kphio
        gammastar = env.gammastar.flatten() # 建议直接使用 PyRealm 计算好的 gammastar
        
        # Calculate Rates
        # Ac
        ac = vcmax_subdaily * (ci_subdaily - gammastar) / (ci_subdaily + kmm)
        ac = subdailyC3.subdaily_Ac
        # J (修正：ppfd -> ppfd_use)
        J = 4 * fkphio_c3 * ppfd_use / np.sqrt(1 + (4 * fkphio_c3 * ppfd_use / jmax_subdaily)**2)
        
        # Aj
        aj = (J / 4) * subdaily_chi.mj
        aj = subdailyC3.subdaily_Aj
        
        # GPP (补充计算，否则下面mask报错)
        gpp = np.minimum(ac, aj)
        gpp = subdailyC3.gpp/12

        # 4) Extract outputs and clean
        # Masking (去除无效值)
 # 4) Extract outputs and clean
        # Masking (remove invalid values)
        mask = np.isfinite(gpp)
        instant_temps = tc_1d[mask]
        
        # Clean specific variables (apply mask first)
        gpp_clean = gpp[mask]
        ac_clean = ac[mask]
        aj_clean = aj[mask]
        jmax25_clean = jmax25_subdaily[mask]
        jmax_clean = jmax_subdaily[mask]
        J_clean = J[mask]  # Clean J variable
        
        # ==========================================================
        # [New Logic 1] Extract Ac and Aj at Growth Temperature
        # ==========================================================
        # Find the index of the growth temperature before removing it
        idx_at_acclim = np.where(np.isclose(instant_temps, acclim_temp))[0]
        
        if len(idx_at_acclim) > 0:
            i_acc = idx_at_acclim[0]
            ac_at_growth_val = ac_clean[i_acc]
            aj_at_growth_val = aj_clean[i_acc]
        else:
            ac_at_growth_val = np.nan
            aj_at_growth_val = np.nan
        # ==========================================================

        # Remove the acclimation-window temperature points
        sel = instant_temps != acclim_temp
        
        # Apply filter to all variables
        final_temps = instant_temps[sel]
        final_gpp = gpp_clean[sel]
        final_ac = ac_clean[sel]
        final_aj = aj_clean[sel]
        final_jmax25 = jmax25_clean[sel]
        final_jmax = jmax_clean[sel]
        final_J = J_clean[sel]  # Filter J variable

        # ==========================================================
        # [Existing Logic] Find Jmax and J at the peak of Aj
        # ==========================================================
        if len(final_aj) > 0:
            idx_peak = np.argmax(final_aj)
            topt_aj_val = final_temps[idx_peak]
            jmax_at_topt_val = final_jmax[idx_peak]
            J_at_topt_val = final_J[idx_peak]
            aj_peak_val = final_aj[idx_peak]
        else:
            topt_aj_val = np.nan
            jmax_at_topt_val = np.nan
            J_at_topt_val = np.nan
            aj_peak_val = np.nan
        
        # Store results
        results_by_light[acclim_ppfd].append({
            'acclim_temp': acclim_temp,
            'instant_temps': final_temps,
            'gpp_values': final_gpp,
            'ac_values': final_ac,
            'aj_values': final_aj,
            'jmax25_values': final_jmax25,
            'jmax_values': final_jmax,
            'J_values': final_J,  # [Added] Store the full J curve
            
            # Scalar results
            'topt_aj': topt_aj_val,
            'jmax_at_topt': jmax_at_topt_val,
            'J_at_topt': J_at_topt_val,
            'aj_peak': aj_peak_val,
            'ac_at_growth': ac_at_growth_val,
            'aj_at_growth': aj_at_growth_val
        })
full_acclimation = results_by_light

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import matplotlib.colors as mcolors

# 假设外部已定义变量：
# n_pts, patm, vpd, co2, fapar, acclim_mask, fsscaler
# memory_effect, calc_kmm, calc_gammastar, OptimalChiPrentice14
# SubdailyPModel, PModel, PModelEnvironment
#fixed gammastar 

acclim_temps = np.arange(0, 41, 2.5)

# ---------- Configuration ----------
light_levels = [100,600, 1200, 1800]      
temps = list(acclim_temps)            

# Colors
norm = mcolors.Normalize(vmin=min(temps), vmax=max(temps))
cmap = cm.get_cmap('coolwarm')

# Store results
results_by_light = {L: [] for L in light_levels}

# ---------- Main loop ----------
for acclim_ppfd in light_levels:
    for acclim_temp in temps:
        # 1) Construct instantaneous forcing
        tc_1d = np.linspace(-10, 60, n_pts).copy()
        patm_use = patm.copy()
        vpd_use  = vpd.copy()
        co2_use  = co2.copy()
        fapar_use = fapar.copy()
        ppfd_use  = np.full(n_pts, 1200.0)    
        tc_1d[acclim_mask] = acclim_temp      
        ppfd_use[acclim_mask] = acclim_ppfd    

        # Daily means
        co2_acclim  = fsscaler.get_daily_means(co2_use)
        vpd_acclim  = fsscaler.get_daily_means(vpd_use)
        patm_acclim = fsscaler.get_daily_means(patm_use)

        # Environment (Subdaily)
        env = PModelEnvironment(tc=tc_1d, patm=patm_use, vpd=vpd_use, co2=co2_use)
        env.aridity_index = np.full(n_pts, 0.0)    
        env.mean_growth_temperature = np.full(n_pts, 25)
        
        # Initial Subdaily Model (for parameters)
        subdailyC3 = SubdailyPModel(
            env=env,
            method_kphio='sandoval',
            reference_kphio=1/9,
            method_optchi='prentice14',
            fapar=fapar_use,
            ppfd=ppfd_use, 
            fs_scaler=fsscaler,
            alpha=1/15,
            allow_holdover=True,
        )

        # 2) Acclimation Model (Daily)
        acclim_env = PModelEnvironment(
            tc=acclim_temp,
            patm=patm_acclim,
            vpd=vpd_acclim,
            co2=co2_acclim
        )
        acclim_env.aridity_index = 0
        acclim_env.mean_growth_temperature = np.array([25])
        #acclim_env.kmm = np.full_like(acclim_env.kmm, 256)
        #acclim_env.fixed_chi = np.full_like(acclim_env.kmm,0.7)
        acclim_env.gammastar = np.full_like(acclim_env.gammastar,4.33)
        pmodel_acclim = PModel(
            env=acclim_env,
            method_kphio='sandoval',
            reference_kphio=1/9,
            method_optchi='prentice14'
        )
        
        # 修正：这里应该使用 env 而不是未定义的 pmodel_env
        pmodel_instant = PModel(
            env=env, 
            method_kphio='sandoval',
            reference_kphio=1/9,
            method_optchi="prentice14",
        )
        
        pmodel_acclim.estimate_productivity(fapar=1, ppfd=acclim_ppfd)
        
        # Constants
        R = 8.3145 
        ha_jmax25 = 43900 
        ha_vcmax25 = 65330 
        
        # 3) Calculate Acclimated Traits (Removing Temp Effect)
        vcmax25_acclim = pmodel_acclim.vcmax * np.exp((ha_vcmax25/R)*(1/(acclim_temp+273.15)-1/298.15))
        jmax25_acclim = pmodel_acclim.jmax * np.exp((ha_jmax25/R)*(1/(acclim_temp+273.15)-1/298.15))

        # Memory Effect
        xi_real = memory_effect(pmodel_acclim.optchi.xi, alpha=1/15)
        vcmax25_real = memory_effect(vcmax25_acclim, alpha=1/15, allow_holdover=True)
        jmax25_real = memory_effect(jmax25_acclim, alpha=1/15, allow_holdover=True)

        # Fill to Subdaily
        vcmax25_subdaily = fsscaler.fill_daily_to_subdaily(vcmax25_real)
        jmax25_subdaily = fsscaler.fill_daily_to_subdaily(jmax25_real)

        # Adjust to Actual Temperature (Subdaily)
        vcmax_subdaily = vcmax25_subdaily * np.exp((ha_vcmax25/R)*(1/298.15-1/(tc_1d+273.15)))
        jmax_subdaily = jmax25_subdaily * np.exp((ha_jmax25/R)*(1/298.15-1/(tc_1d+273.15)))
        
        # Instantaneous Chi
        xi_subdaily = fsscaler.fill_daily_to_subdaily(xi_real)
        # Calculate optimal chi
        # 修正：这里同样应该使用 env
        subdaily_chi = OptimalChiPrentice14(env=env) 
        subdaily_chi.estimate_chi(xi_values=xi_subdaily)
        
        ci_subdaily = subdaily_chi.ci
        # kmm = calc_kmm(tc_1d, patm.transpose()) # 如果 env 里已经有了，可以直接用 env.kmm
        kmm = env.kmm.flatten() # 建议直接使用 PyRealm 计算好的 kmm
        
        fkphio_c3 = subdailyC3.kphio.kphio
        gammastar = env.gammastar.flatten() # 建议直接使用 PyRealm 计算好的 gammastar
        
        # Calculate Rates
        # Ac
        #ac = subdailyC3.subdaily_Ac
        ac =vcmax_subdaily * (ci_subdaily - gammastar) / (ci_subdaily + kmm)
        
        # J (修正：ppfd -> ppfd_use)
        J = 4 * fkphio_c3 * ppfd_use / np.sqrt(1 + (4 * fkphio_c3 * ppfd_use / jmax_subdaily)**2)
        
        # Aj
        aj = (J / 4) * subdaily_chi.mj
        # aj = subdailyC3.subdaily_Aj
        
        # GPP (补充计算，否则下面mask报错)
        gpp = np.minimum(ac, aj)

        # 4) Extract outputs and clean
# 4) Extract outputs and clean
        # Masking (去除无效值)
 # 4) Extract outputs and clean
        # Masking (remove invalid values)
        mask = np.isfinite(gpp)
        instant_temps = tc_1d[mask]
        
        # Clean specific variables (apply mask first)
        gpp_clean = gpp[mask]
        ac_clean = ac[mask]
        aj_clean = aj[mask]
        jmax25_clean = jmax25_subdaily[mask]
        jmax_clean = jmax_subdaily[mask]
        J_clean = J[mask]  # Clean J variable
        
        # ==========================================================
        # [New Logic 1] Extract Ac and Aj at Growth Temperature
        # ==========================================================
        # Find the index of the growth temperature before removing it
        idx_at_acclim = np.where(np.isclose(instant_temps, acclim_temp))[0]
        
        if len(idx_at_acclim) > 0:
            i_acc = idx_at_acclim[0]
            ac_at_growth_val = ac_clean[i_acc]
            aj_at_growth_val = aj_clean[i_acc]
        else:
            ac_at_growth_val = np.nan
            aj_at_growth_val = np.nan
        # ==========================================================

        # Remove the acclimation-window temperature points
        sel = instant_temps != acclim_temp
        
        # Apply filter to all variables
        final_temps = instant_temps[sel]
        final_gpp = gpp_clean[sel]
        final_ac = ac_clean[sel]
        final_aj = aj_clean[sel]
        final_jmax25 = jmax25_clean[sel]
        final_jmax = jmax_clean[sel]
        final_J = J_clean[sel]  # Filter J variable

        # ==========================================================
        # [Existing Logic] Find Jmax and J at the peak of Aj
        # ==========================================================
        if len(final_aj) > 0:
            idx_peak = np.argmax(final_aj)
            topt_aj_val = final_temps[idx_peak]
            jmax_at_topt_val = final_jmax[idx_peak]
            J_at_topt_val = final_J[idx_peak]
            aj_peak_val = final_aj[idx_peak]
        else:
            topt_aj_val = np.nan
            jmax_at_topt_val = np.nan
            J_at_topt_val = np.nan
            aj_peak_val = np.nan
        
        # Store results
        results_by_light[acclim_ppfd].append({
            'acclim_temp': acclim_temp,
            'instant_temps': final_temps,
            'gpp_values': final_gpp,
            'ac_values': final_ac,
            'aj_values': final_aj,
            'jmax25_values': final_jmax25,
            'jmax_values': final_jmax,
            'J_values': final_J,  # [Added] Store the full J curve
            
            # Scalar results
            'topt_aj': topt_aj_val,
            'jmax_at_topt': jmax_at_topt_val,
            'J_at_topt': J_at_topt_val,
            'aj_peak': aj_peak_val,
            'ac_at_growth': ac_at_growth_val,
            'aj_at_growth': aj_at_growth_val
        })
res_gamma = results_by_light.copy()
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import matplotlib.colors as mcolors

# 假设外部已定义变量：
# n_pts, patm, vpd, co2, fapar, acclim_mask, fsscaler
# memory_effect, calc_kmm, calc_gammastar, OptimalChiPrentice14
# SubdailyPModel, PModel, PModelEnvironment
#fixed chi

# ---------- Configuration ----------
light_levels = [250,600, 1200, 1800]      
temps = list(acclim_temps)            

# Colors
norm = mcolors.Normalize(vmin=min(temps), vmax=max(temps))
cmap = cm.get_cmap('coolwarm')

# Store results
results_by_light = {L: [] for L in light_levels}

# ---------- Main loop ----------
for acclim_ppfd in light_levels:
    for acclim_temp in temps:
        # 1) Construct instantaneous forcing
        tc_1d = np.linspace(-10, 60, n_pts).copy()
        patm_use = patm.copy()
        vpd_use  = vpd.copy()
        co2_use  = co2.copy()
        fapar_use = fapar.copy()
        ppfd_use  = np.full(n_pts, 1200.0)    
        tc_1d[acclim_mask] = acclim_temp      
        ppfd_use[acclim_mask] = acclim_ppfd    

        # Daily means
        co2_acclim  = fsscaler.get_daily_means(co2_use)
        vpd_acclim  = fsscaler.get_daily_means(vpd_use)
        patm_acclim = fsscaler.get_daily_means(patm_use)

        # Environment (Subdaily)
        env = PModelEnvironment(tc=tc_1d, patm=patm_use, vpd=vpd_use, co2=co2_use)
        env.aridity_index = np.full(n_pts, 0.0)    
        env.mean_growth_temperature = np.full(n_pts, 25)
        
        # Initial Subdaily Model (for parameters)
        subdailyC3 = SubdailyPModel(
            env=env,
            method_kphio='sandoval',
            reference_kphio=1/9,
            method_optchi='prentice14',
            fapar=fapar_use,
            ppfd=ppfd_use, 
            fs_scaler=fsscaler,
            alpha=1/15,
            allow_holdover=True,
        )

        # 2) Acclimation Model (Daily)
        acclim_env = PModelEnvironment(
            tc=acclim_temp,
            patm=patm_acclim,
            vpd=vpd_acclim,
            co2=co2_acclim
        )
        acclim_env.aridity_index = 0
        acclim_env.mean_growth_temperature = np.array([25])
        #acclim_env.kmm = np.full_like(acclim_env.kmm, 256)
        acclim_env.fixed_chi = np.full_like(acclim_env.kmm,0.7)
        #acclim_env.gammastar = np.full_like(acclim_env.gammastar,4.33)
        pmodel_acclim = PModel(
            env=acclim_env,
            method_kphio='sandoval',
            reference_kphio=1/9,
            method_optchi='fixed'
        )
        
        # 修正：这里应该使用 env 而不是未定义的 pmodel_env
        pmodel_instant = PModel(
            env=env, 
            method_kphio='sandoval',
            reference_kphio=1/9,
            method_optchi="prentice14",
        )
        
        pmodel_acclim.estimate_productivity(fapar=1, ppfd=acclim_ppfd)
        
        # Constants
        R = 8.3145 
        ha_jmax25 = 43900 
        ha_vcmax25 = 65330 
        
        # 3) Calculate Acclimated Traits (Removing Temp Effect)
        vcmax25_acclim = pmodel_acclim.vcmax * np.exp((ha_vcmax25/R)*(1/(acclim_temp+273.15)-1/298.15))
        jmax25_acclim = pmodel_acclim.jmax * np.exp((ha_jmax25/R)*(1/(acclim_temp+273.15)-1/298.15))

        # Memory Effect
        xi_real = memory_effect(pmodel_acclim.optchi.xi, alpha=1/15)
        vcmax25_real = memory_effect(vcmax25_acclim, alpha=1/15, allow_holdover=True)
        jmax25_real = memory_effect(jmax25_acclim, alpha=1/15, allow_holdover=True)

        # Fill to Subdaily
        vcmax25_subdaily = fsscaler.fill_daily_to_subdaily(vcmax25_real)
        jmax25_subdaily = fsscaler.fill_daily_to_subdaily(jmax25_real)

        # Adjust to Actual Temperature (Subdaily)
        vcmax_subdaily = vcmax25_subdaily * np.exp((ha_vcmax25/R)*(1/298.15-1/(tc_1d+273.15)))
        jmax_subdaily = jmax25_subdaily * np.exp((ha_jmax25/R)*(1/298.15-1/(tc_1d+273.15)))
        
        # Instantaneous Chi
        xi_subdaily = fsscaler.fill_daily_to_subdaily(xi_real)
        # Calculate optimal chi
        # 修正：这里同样应该使用 env
        subdaily_chi = OptimalChiPrentice14(env=env) 
        subdaily_chi.estimate_chi(xi_values=xi_subdaily)
        
        ci_subdaily = subdaily_chi.ci
        # kmm = calc_kmm(tc_1d, patm.transpose()) # 如果 env 里已经有了，可以直接用 env.kmm
        kmm = env.kmm.flatten() # 建议直接使用 PyRealm 计算好的 kmm
        
        fkphio_c3 = subdailyC3.kphio.kphio
        gammastar = env.gammastar.flatten() # 建议直接使用 PyRealm 计算好的 gammastar
        
        # Calculate Rates
        # Ac
        ac =vcmax_subdaily * (ci_subdaily - gammastar) / (ci_subdaily + kmm)
        
        # J (修正：ppfd -> ppfd_use)
        J = 4 * fkphio_c3 * ppfd_use / np.sqrt(1 + (4 * fkphio_c3 * ppfd_use / jmax_subdaily)**2)
        
        # Aj
        aj = (J / 4) * subdaily_chi.mj
        # aj = subdailyC3.subdaily_Aj
        
        # GPP (补充计算，否则下面mask报错)
        gpp = np.minimum(ac, aj)

        # 4) Extract outputs and clean
# 4) Extract outputs and clean
        # Masking (去除无效值)
 # 4) Extract outputs and clean
        # Masking (remove invalid values)
        mask = np.isfinite(gpp)
        instant_temps = tc_1d[mask]
        
        # Clean specific variables (apply mask first)
        gpp_clean = gpp[mask]
        ac_clean = ac[mask]
        aj_clean = aj[mask]
        jmax25_clean = jmax25_subdaily[mask]
        jmax_clean = jmax_subdaily[mask]
        J_clean = J[mask]  # Clean J variable
        
        # ==========================================================
        # [New Logic 1] Extract Ac and Aj at Growth Temperature
        # ==========================================================
        # Find the index of the growth temperature before removing it
        idx_at_acclim = np.where(np.isclose(instant_temps, acclim_temp))[0]
        
        if len(idx_at_acclim) > 0:
            i_acc = idx_at_acclim[0]
            ac_at_growth_val = ac_clean[i_acc]
            aj_at_growth_val = aj_clean[i_acc]
        else:
            ac_at_growth_val = np.nan
            aj_at_growth_val = np.nan
        # ==========================================================

        # Remove the acclimation-window temperature points
        sel = instant_temps != acclim_temp
        
        # Apply filter to all variables
        final_temps = instant_temps[sel]
        final_gpp = gpp_clean[sel]
        final_ac = ac_clean[sel]
        final_aj = aj_clean[sel]
        final_jmax25 = jmax25_clean[sel]
        final_jmax = jmax_clean[sel]
        final_J = J_clean[sel]  # Filter J variable

        # ==========================================================
        # [Existing Logic] Find Jmax and J at the peak of Aj
        # ==========================================================
        if len(final_aj) > 0:
            idx_peak = np.argmax(final_aj)
            topt_aj_val = final_temps[idx_peak]
            jmax_at_topt_val = final_jmax[idx_peak]
            J_at_topt_val = final_J[idx_peak]
            aj_peak_val = final_aj[idx_peak]
        else:
            topt_aj_val = np.nan
            jmax_at_topt_val = np.nan
            J_at_topt_val = np.nan
            aj_peak_val = np.nan
        
        # Store results
        results_by_light[acclim_ppfd].append({
            'acclim_temp': acclim_temp,
            'instant_temps': final_temps,
            'gpp_values': final_gpp,
            'ac_values': final_ac,
            'aj_values': final_aj,
            'jmax25_values': final_jmax25,
            'jmax_values': final_jmax,
            'J_values': final_J,  # [Added] Store the full J curve
            
            # Scalar results
            'topt_aj': topt_aj_val,
            'jmax_at_topt': jmax_at_topt_val,
            'J_at_topt': J_at_topt_val,
            'aj_peak': aj_peak_val,
            'ac_at_growth': ac_at_growth_val,
            'aj_at_growth': aj_at_growth_val
        })
res_chi = results_by_light.copy()

# 假设外部已定义变量：
# n_pts, patm, vpd, co2, fapar, acclim_mask, fsscaler
# memory_effect, calc_kmm, calc_gammastar, OptimalChiPrentice14
# SubdailyPModel, PModel, PModelEnvironment
#fixed phi0

# ---------- Configuration ----------
light_levels = [100,600, 1200, 1800]      
temps = list(acclim_temps)            

# Colors
norm = mcolors.Normalize(vmin=min(temps), vmax=max(temps))
cmap = cm.get_cmap('coolwarm')

# Store results
results_by_light = {L: [] for L in light_levels}

# ---------- Main loop ----------
for acclim_ppfd in light_levels:
    for acclim_temp in temps:
        # 1) Construct instantaneous forcing
        tc_1d = np.linspace(-10, 60, n_pts).copy()
        patm_use = patm.copy()
        vpd_use  = vpd.copy()
        co2_use  = co2.copy()
        fapar_use = fapar.copy()
        ppfd_use  = np.full(n_pts, 1200.0)    
        tc_1d[acclim_mask] = acclim_temp      
        ppfd_use[acclim_mask] = acclim_ppfd    

        # Daily means
        co2_acclim  = fsscaler.get_daily_means(co2_use)
        vpd_acclim  = fsscaler.get_daily_means(vpd_use)
        patm_acclim = fsscaler.get_daily_means(patm_use)

        # Environment (Subdaily)
        env = PModelEnvironment(tc=tc_1d, patm=patm_use, vpd=vpd_use, co2=co2_use)
        env.aridity_index = np.full(n_pts, 0.0)    
        env.mean_growth_temperature = np.full(n_pts, 25)
        
        # Initial Subdaily Model (for parameters)
        subdailyC3 = SubdailyPModel(
            env=env,
            method_kphio='sandoval',
            reference_kphio=1/9,
            method_optchi='prentice14',
            fapar=fapar_use,
            ppfd=ppfd_use, 
            fs_scaler=fsscaler,
            alpha=1/15,
            allow_holdover=True,
        )

        # 2) Acclimation Model (Daily)
        acclim_env = PModelEnvironment(
            tc=acclim_temp,
            patm=patm_acclim,
            vpd=vpd_acclim,
            co2=co2_acclim
        )
        acclim_env.aridity_index = 0
        acclim_env.mean_growth_temperature = np.array([25])
        #acclim_env.kmm = np.full_like(acclim_env.kmm, 256)
        #acclim_env.fixed_chi = np.full_like(acclim_env.kmm,0.7)
        #acclim_env.gammastar = np.full_like(acclim_env.gammastar,4.33)
        pmodel_acclim = PModel(
            env=acclim_env,
            method_kphio='fixed',
            reference_kphio=1/9,
            method_optchi='prentice14'
        )
        
        # 修正：这里应该使用 env 而不是未定义的 pmodel_env
        pmodel_instant = PModel(
            env=env, 
            method_kphio='sandoval',
            reference_kphio=1/9,
            method_optchi="prentice14",
        )
        
        pmodel_acclim.estimate_productivity(fapar=1, ppfd=acclim_ppfd)
        
        # Constants
        R = 8.3145 
        ha_jmax25 = 43900 
        ha_vcmax25 = 65330 
        
        # 3) Calculate Acclimated Traits (Removing Temp Effect)
        vcmax25_acclim = pmodel_acclim.vcmax * np.exp((ha_vcmax25/R)*(1/(acclim_temp+273.15)-1/298.15))
        jmax25_acclim = pmodel_acclim.jmax * np.exp((ha_jmax25/R)*(1/(acclim_temp+273.15)-1/298.15))

        # Memory Effect
        xi_real = memory_effect(pmodel_acclim.optchi.xi, alpha=1/15)
        vcmax25_real = memory_effect(vcmax25_acclim, alpha=1/15, allow_holdover=True)
        jmax25_real = memory_effect(jmax25_acclim, alpha=1/15, allow_holdover=True)

        # Fill to Subdaily
        vcmax25_subdaily = fsscaler.fill_daily_to_subdaily(vcmax25_real)
        jmax25_subdaily = fsscaler.fill_daily_to_subdaily(jmax25_real)

        # Adjust to Actual Temperature (Subdaily)
        vcmax_subdaily = vcmax25_subdaily * np.exp((ha_vcmax25/R)*(1/298.15-1/(tc_1d+273.15)))
        jmax_subdaily = jmax25_subdaily * np.exp((ha_jmax25/R)*(1/298.15-1/(tc_1d+273.15)))
        
        # Instantaneous Chi
        xi_subdaily = fsscaler.fill_daily_to_subdaily(xi_real)
        # Calculate optimal chi
        # 修正：这里同样应该使用 env
        subdaily_chi = OptimalChiPrentice14(env=env) 
        subdaily_chi.estimate_chi(xi_values=xi_subdaily)
        
        ci_subdaily = subdaily_chi.ci
        # kmm = calc_kmm(tc_1d, patm.transpose()) # 如果 env 里已经有了，可以直接用 env.kmm
        kmm = env.kmm.flatten() # 建议直接使用 PyRealm 计算好的 kmm
        
        fkphio_c3 = subdailyC3.kphio.kphio
        gammastar = env.gammastar.flatten() # 建议直接使用 PyRealm 计算好的 gammastar
        
        # Calculate Rates
        # Ac
        ac =vcmax_subdaily * (ci_subdaily - gammastar) / (ci_subdaily + kmm)
        
        # J (修正：ppfd -> ppfd_use)
        J = 4 * fkphio_c3 * ppfd_use / np.sqrt(1 + (4 * fkphio_c3 * ppfd_use / jmax_subdaily)**2)
        
        # Aj
        aj = (J / 4) * subdaily_chi.mj
        # aj = subdailyC3.subdaily_Aj
        
        # GPP (补充计算，否则下面mask报错)
        gpp = np.minimum(ac, aj)

        # 4) Extract outputs and clean
# 4) Extract outputs and clean
        # Masking (去除无效值)
 # 4) Extract outputs and clean
        # Masking (remove invalid values)
        mask = np.isfinite(gpp)
        instant_temps = tc_1d[mask]
        
        # Clean specific variables (apply mask first)
        gpp_clean = gpp[mask]
        ac_clean = ac[mask]
        aj_clean = aj[mask]
        jmax25_clean = jmax25_subdaily[mask]
        jmax_clean = jmax_subdaily[mask]
        J_clean = J[mask]  # Clean J variable
        
        # ==========================================================
        # [New Logic 1] Extract Ac and Aj at Growth Temperature
        # ==========================================================
        # Find the index of the growth temperature before removing it
        idx_at_acclim = np.where(np.isclose(instant_temps, acclim_temp))[0]
        
        if len(idx_at_acclim) > 0:
            i_acc = idx_at_acclim[0]
            ac_at_growth_val = ac_clean[i_acc]
            aj_at_growth_val = aj_clean[i_acc]
        else:
            ac_at_growth_val = np.nan
            aj_at_growth_val = np.nan
        # ==========================================================

        # Remove the acclimation-window temperature points
        sel = instant_temps != acclim_temp
        
        # Apply filter to all variables
        final_temps = instant_temps[sel]
        final_gpp = gpp_clean[sel]
        final_ac = ac_clean[sel]
        final_aj = aj_clean[sel]
        final_jmax25 = jmax25_clean[sel]
        final_jmax = jmax_clean[sel]
        final_J = J_clean[sel]  # Filter J variable

        # ==========================================================
        # [Existing Logic] Find Jmax and J at the peak of Aj
        # ==========================================================
        if len(final_aj) > 0:
            idx_peak = np.argmax(final_aj)
            topt_aj_val = final_temps[idx_peak]
            jmax_at_topt_val = final_jmax[idx_peak]
            J_at_topt_val = final_J[idx_peak]
            aj_peak_val = final_aj[idx_peak]
        else:
            topt_aj_val = np.nan
            jmax_at_topt_val = np.nan
            J_at_topt_val = np.nan
            aj_peak_val = np.nan
        
        # Store results
        results_by_light[acclim_ppfd].append({
            'acclim_temp': acclim_temp,
            'instant_temps': final_temps,
            'gpp_values': final_gpp,
            'ac_values': final_ac,
            'aj_values': final_aj,
            'jmax25_values': final_jmax25,
            'jmax_values': final_jmax,
            'J_values': final_J,  # [Added] Store the full J curve
            
            # Scalar results
            'topt_aj': topt_aj_val,
            'jmax_at_topt': jmax_at_topt_val,
            'J_at_topt': J_at_topt_val,
            'aj_peak': aj_peak_val,
            'ac_at_growth': ac_at_growth_val,
            'aj_at_growth': aj_at_growth_val
        })
res_phi = results_by_light.copy()
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import matplotlib.colors as mcolors

# 假设外部已定义变量：
# n_pts, patm, vpd, co2, fapar, acclim_mask, fsscaler
# memory_effect, calc_kmm, calc_gammastar, OptimalChiPrentice14
# SubdailyPModel, PModel, PModelEnvironment
#fixed gammastar 


# ---------- Configuration ----------
light_levels = [100,600, 1200, 1800]      
temps = list(acclim_temps)            

# Colors
norm = mcolors.Normalize(vmin=min(temps), vmax=max(temps))
cmap = cm.get_cmap('coolwarm')

# Store results
results_by_light = {L: [] for L in light_levels}

# ---------- Main loop ----------
for acclim_ppfd in light_levels:
    for acclim_temp in temps:
        # 1) Construct instantaneous forcing
        tc_1d = np.linspace(-10, 60, n_pts).copy()
        patm_use = patm.copy()
        vpd_use  = vpd.copy()
        co2_use  = co2.copy()
        fapar_use = fapar.copy()
        ppfd_use  = np.full(n_pts, 1200.0)    
        tc_1d[acclim_mask] = acclim_temp      
        ppfd_use[acclim_mask] = acclim_ppfd    

        # Daily means
        co2_acclim  = fsscaler.get_daily_means(co2_use)
        vpd_acclim  = fsscaler.get_daily_means(vpd_use)
        patm_acclim = fsscaler.get_daily_means(patm_use)

        # Environment (Subdaily)
        env = PModelEnvironment(tc=tc_1d, patm=patm_use, vpd=vpd_use, co2=co2_use)
        env.aridity_index = np.full(n_pts, 0.0)    
        env.mean_growth_temperature = np.full(n_pts, 25)
        
        # Initial Subdaily Model (for parameters)
        subdailyC3 = SubdailyPModel(
            env=env,
            method_kphio='sandoval',
            reference_kphio=1/9,
            method_optchi='prentice14',
            fapar=fapar_use,
            ppfd=ppfd_use, 
            fs_scaler=fsscaler,
            alpha=1/15,
            allow_holdover=True,
        )

        # 2) Acclimation Model (Daily)
        acclim_env = PModelEnvironment(
            tc=acclim_temp,
            patm=patm_acclim,
            vpd=vpd_acclim,
            co2=co2_acclim
        )
        acclim_env.aridity_index = 0
        acclim_env.mean_growth_temperature = np.array([25])
        #acclim_env.kmm = np.full_like(acclim_env.kmm, 256)
        #acclim_env.fixed_chi = np.full_like(acclim_env.kmm,0.7)
        acclim_env.gammastar = np.full_like(acclim_env.gammastar,4.33)
        pmodel_acclim = PModel(
            env=acclim_env,
            method_kphio='sandoval',
            reference_kphio=1/9,
            method_optchi='prentice14'
        )
        
        # 修正：这里应该使用 env 而不是未定义的 pmodel_env
        pmodel_instant = PModel(
            env=env, 
            method_kphio='sandoval',
            reference_kphio=1/9,
            method_optchi="prentice14",
        )
        
        pmodel_acclim.estimate_productivity(fapar=1, ppfd=acclim_ppfd)
        
        # Constants
        R = 8.3145 
        ha_jmax25 = 43900 
        ha_vcmax25 = 65330 
        
        # 3) Calculate Acclimated Traits (Removing Temp Effect)
        vcmax25_acclim = pmodel_acclim.vcmax * np.exp((ha_vcmax25/R)*(1/(acclim_temp+273.15)-1/298.15))
        jmax25_acclim = pmodel_acclim.jmax * np.exp((0/R)*(1/(acclim_temp+273.15)-1/298.15))

        # Memory Effect
        xi_real = memory_effect(pmodel_acclim.optchi.xi, alpha=1/15)
        vcmax25_real = memory_effect(vcmax25_acclim, alpha=1/15, allow_holdover=True)
        jmax25_real = memory_effect(jmax25_acclim, alpha=1/15, allow_holdover=True)

        # Fill to Subdaily
        vcmax25_subdaily = fsscaler.fill_daily_to_subdaily(vcmax25_real)
        jmax25_subdaily = fsscaler.fill_daily_to_subdaily(jmax25_real)

        # Adjust to Actual Temperature (Subdaily)
        vcmax_subdaily = vcmax25_subdaily * np.exp((ha_vcmax25/R)*(1/298.15-1/(tc_1d+273.15)))
        jmax_subdaily = jmax25_subdaily * np.exp((ha_jmax25/R)*(1/298.15-1/(tc_1d+273.15)))
        
        # Instantaneous Chi
        xi_subdaily = fsscaler.fill_daily_to_subdaily(xi_real)
        # Calculate optimal chi
        # 修正：这里同样应该使用 env
        subdaily_chi = OptimalChiPrentice14(env=env) 
        subdaily_chi.estimate_chi(xi_values=xi_subdaily)
        
        ci_subdaily = subdaily_chi.ci
        # kmm = calc_kmm(tc_1d, patm.transpose()) # 如果 env 里已经有了，可以直接用 env.kmm
        kmm = env.kmm.flatten() # 建议直接使用 PyRealm 计算好的 kmm
        
        fkphio_c3 = subdailyC3.kphio.kphio
        gammastar = env.gammastar.flatten() # 建议直接使用 PyRealm 计算好的 gammastar
        
        # Calculate Rates
        # Ac
        #ac = subdailyC3.subdaily_Ac
        ac =vcmax_subdaily * (ci_subdaily - gammastar) / (ci_subdaily + kmm)
        
        # J (修正：ppfd -> ppfd_use)
        J = 4 * fkphio_c3 * ppfd_use / np.sqrt(1 + (4 * fkphio_c3 * ppfd_use / jmax_subdaily)**2)
        
        # Aj
        aj = (J / 4) * subdaily_chi.mj
        # aj = subdailyC3.subdaily_Aj
        
        # GPP (补充计算，否则下面mask报错)
        gpp = np.minimum(ac, aj)

        # 4) Extract outputs and clean
# 4) Extract outputs and clean
        # Masking (去除无效值)
 # 4) Extract outputs and clean
        # Masking (remove invalid values)
        mask = np.isfinite(gpp)
        instant_temps = tc_1d[mask]
        
        # Clean specific variables (apply mask first)
        gpp_clean = gpp[mask]
        ac_clean = ac[mask]
        aj_clean = aj[mask]
        jmax25_clean = jmax25_subdaily[mask]
        jmax_clean = jmax_subdaily[mask]
        J_clean = J[mask]  # Clean J variable
        
        # ==========================================================
        # [New Logic 1] Extract Ac and Aj at Growth Temperature
        # ==========================================================
        # Find the index of the growth temperature before removing it
        idx_at_acclim = np.where(np.isclose(instant_temps, acclim_temp))[0]
        
        if len(idx_at_acclim) > 0:
            i_acc = idx_at_acclim[0]
            ac_at_growth_val = ac_clean[i_acc]
            aj_at_growth_val = aj_clean[i_acc]
        else:
            ac_at_growth_val = np.nan
            aj_at_growth_val = np.nan
        # ==========================================================

        # Remove the acclimation-window temperature points
        sel = instant_temps != acclim_temp
        
        # Apply filter to all variables
        final_temps = instant_temps[sel]
        final_gpp = gpp_clean[sel]
        final_ac = ac_clean[sel]
        final_aj = aj_clean[sel]
        final_jmax25 = jmax25_clean[sel]
        final_jmax = jmax_clean[sel]
        final_J = J_clean[sel]  # Filter J variable

        # ==========================================================
        # [Existing Logic] Find Jmax and J at the peak of Aj
        # ==========================================================
        if len(final_aj) > 0:
            idx_peak = np.argmax(final_aj)
            topt_aj_val = final_temps[idx_peak]
            jmax_at_topt_val = final_jmax[idx_peak]
            J_at_topt_val = final_J[idx_peak]
            aj_peak_val = final_aj[idx_peak]
        else:
            topt_aj_val = np.nan
            jmax_at_topt_val = np.nan
            J_at_topt_val = np.nan
            aj_peak_val = np.nan
        
        # Store results
        results_by_light[acclim_ppfd].append({
            'acclim_temp': acclim_temp,
            'instant_temps': final_temps,
            'gpp_values': final_gpp,
            'ac_values': final_ac,
            'aj_values': final_aj,
            'jmax25_values': final_jmax25,
            'jmax_values': final_jmax,
            'J_values': final_J,  # [Added] Store the full J curve
            
            # Scalar results
            'topt_aj': topt_aj_val,
            'jmax_at_topt': jmax_at_topt_val,
            'J_at_topt': J_at_topt_val,
            'aj_peak': aj_peak_val,
            'ac_at_growth': ac_at_growth_val,
            'aj_at_growth': aj_at_growth_val
        })
res_energy = results_by_light.copy()
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import matplotlib.colors as mcolors

# 假设外部已定义变量：
# n_pts, patm, vpd, co2, fapar, acclim_mask, fsscaler
# memory_effect, calc_kmm, calc_gammastar, OptimalChiPrentice14
# SubdailyPModel, PModel, PModelEnvironment
#cstar 0

# ---------- Configuration ----------
light_levels = [100,600, 1200, 1800]      
temps = list(acclim_temps)            

# Colors
norm = mcolors.Normalize(vmin=min(temps), vmax=max(temps))
cmap = cm.get_cmap('coolwarm')

# Store results
results_by_light = {L: [] for L in light_levels}

# ---------- Main loop ----------
for acclim_ppfd in light_levels:
    for acclim_temp in temps:
        # 1) Construct instantaneous forcing
        tc_1d = np.linspace(-10, 60, n_pts).copy()
        patm_use = patm.copy()
        vpd_use  = vpd.copy()
        co2_use  = co2.copy()
        fapar_use = fapar.copy()
        ppfd_use  = np.full(n_pts, 1200.0)    
        tc_1d[acclim_mask] = acclim_temp      
        ppfd_use[acclim_mask] = acclim_ppfd    

        # Daily means
        co2_acclim  = fsscaler.get_daily_means(co2_use)
        vpd_acclim  = fsscaler.get_daily_means(vpd_use)
        patm_acclim = fsscaler.get_daily_means(patm_use)

        # Environment (Subdaily)
        env = PModelEnvironment(tc=tc_1d, patm=patm_use, vpd=vpd_use, co2=co2_use)
        env.aridity_index = np.full(n_pts, 0.0)    
        env.mean_growth_temperature = np.full(n_pts, 25)
        
        # Initial Subdaily Model (for parameters)
        subdailyC3 = SubdailyPModel(
            env=env,
            method_kphio='sandoval',
            reference_kphio=1/9,
            method_optchi='prentice14',
            fapar=fapar_use,
            ppfd=ppfd_use, 
            fs_scaler=fsscaler,
            alpha=1/15,
            allow_holdover=True,
        )

        # 2) Acclimation Model (Daily)
        acclim_env = PModelEnvironment(
            tc=acclim_temp,
            patm=patm_acclim,
            vpd=vpd_acclim,
            co2=co2_acclim
        )
        acclim_env.aridity_index = 0
        acclim_env.mean_growth_temperature = np.array([25])
        acclim_env.pmodel_const = pmodel_const

        #acclim_env.kmm = np.full_like(acclim_env.kmm, 256)
        #acclim_env.fixed_chi = np.full_like(acclim_env.kmm,0.7)
        #acclim_env.gammastar = np.full_like(acclim_env.gammastar,4.33)
        pmodel_acclim = PModel(
            env=acclim_env,
            method_kphio='sandoval',
            reference_kphio=1/9,
            method_optchi='prentice14',
        )
        
        # 修正：这里应该使用 env 而不是未定义的 pmodel_env
        pmodel_instant = PModel(
            env=env, 
            method_kphio='sandoval',
            reference_kphio=1/9,
            method_optchi="prentice14",
        )
        
        pmodel_acclim.estimate_productivity(fapar=1, ppfd=acclim_ppfd)
        
        # Constants
        R = 8.3145 
        ha_jmax25 = 43900 
        ha_vcmax25 = 65330 
        
        # 3) Calculate Acclimated Traits (Removing Temp Effect)
        vcmax25_acclim = pmodel_acclim.vcmax * np.exp((ha_vcmax25/R)*(1/(acclim_temp+273.15)-1/298.15))
        jmax25_acclim = pmodel_acclim.jmax * np.exp((ha_jmax25/R)*(1/(acclim_temp+273.15)-1/298.15))

        # Memory Effect
        xi_real = memory_effect(pmodel_acclim.optchi.xi, alpha=1/15)
        vcmax25_real = memory_effect(vcmax25_acclim, alpha=1/15, allow_holdover=True)
        jmax25_real = memory_effect(jmax25_acclim, alpha=1/15, allow_holdover=True)

        # Fill to Subdaily
        vcmax25_subdaily = fsscaler.fill_daily_to_subdaily(vcmax25_real)
        jmax25_subdaily = fsscaler.fill_daily_to_subdaily(jmax25_real)

        # Adjust to Actual Temperature (Subdaily)
        vcmax_subdaily = vcmax25_subdaily * np.exp((ha_vcmax25/R)*(1/298.15-1/(tc_1d+273.15)))
        jmax_subdaily = jmax25_subdaily * np.exp((ha_jmax25/R)*(1/298.15-1/(tc_1d+273.15)))
        
        # Instantaneous Chi
        xi_subdaily = fsscaler.fill_daily_to_subdaily(xi_real)
        # Calculate optimal chi
        # 修正：这里同样应该使用 env
        subdaily_chi = OptimalChiPrentice14(env=env) 
        subdaily_chi.estimate_chi(xi_values=xi_subdaily)
        
        ci_subdaily = subdaily_chi.ci
        # kmm = calc_kmm(tc_1d, patm.transpose()) # 如果 env 里已经有了，可以直接用 env.kmm
        kmm = env.kmm.flatten() # 建议直接使用 PyRealm 计算好的 kmm
        
        fkphio_c3 = subdailyC3.kphio.kphio
        gammastar = env.gammastar.flatten() # 建议直接使用 PyRealm 计算好的 gammastar
        
        # Calculate Rates
        # Ac
        #ac = subdailyC3.subdaily_Ac
        ac =vcmax_subdaily * (ci_subdaily - gammastar) / (ci_subdaily + kmm)
        
        
        # J (修正：ppfd -> ppfd_use)
        J = 4 * fkphio_c3 * ppfd_use / np.sqrt(1 + (4 * fkphio_c3 * ppfd_use / jmax_subdaily)**2)
        
        # Aj
        aj = (J / 4) * subdaily_chi.mj
        # aj = subdailyC3.subdaily_Aj
        
        # GPP (补充计算，否则下面mask报错)
        gpp = np.minimum(ac, aj)

        # 4) Extract outputs and clean
# 4) Extract outputs and clean
        # Masking (去除无效值)
 # 4) Extract outputs and clean
        # Masking (remove invalid values)
        mask = np.isfinite(gpp)
        instant_temps = tc_1d[mask]
        
        # Clean specific variables (apply mask first)
        gpp_clean = gpp[mask]
        ac_clean = ac[mask]
        aj_clean = aj[mask]
        jmax25_clean = jmax25_subdaily[mask]
        jmax_clean = jmax_subdaily[mask]
        J_clean = J[mask]  # Clean J variable
        
        # ==========================================================
        # [New Logic 1] Extract Ac and Aj at Growth Temperature
        # ==========================================================
        # Find the index of the growth temperature before removing it
        idx_at_acclim = np.where(np.isclose(instant_temps, acclim_temp))[0]
        
        if len(idx_at_acclim) > 0:
            i_acc = idx_at_acclim[0]
            ac_at_growth_val = ac_clean[i_acc]
            aj_at_growth_val = aj_clean[i_acc]
        else:
            ac_at_growth_val = np.nan
            aj_at_growth_val = np.nan
        # ==========================================================

        # Remove the acclimation-window temperature points
        sel = instant_temps != acclim_temp
        
        # Apply filter to all variables
        final_temps = instant_temps[sel]
        final_gpp = gpp_clean[sel]
        final_ac = ac_clean[sel]
        final_aj = aj_clean[sel]
        final_jmax25 = jmax25_clean[sel]
        final_jmax = jmax_clean[sel]
        final_J = J_clean[sel]  # Filter J variable

        # ==========================================================
        # [Existing Logic] Find Jmax and J at the peak of Aj
        # ==========================================================
        if len(final_aj) > 0:
            idx_peak = np.argmax(final_aj)
            topt_aj_val = final_temps[idx_peak]
            jmax_at_topt_val = final_jmax[idx_peak]
            J_at_topt_val = final_J[idx_peak]
            aj_peak_val = final_aj[idx_peak]
        else:
            topt_aj_val = np.nan
            jmax_at_topt_val = np.nan
            J_at_topt_val = np.nan
            aj_peak_val = np.nan
        
        # Store results
        results_by_light[acclim_ppfd].append({
            'acclim_temp': acclim_temp,
            'instant_temps': final_temps,
            'gpp_values': final_gpp,
            'ac_values': final_ac,
            'aj_values': final_aj,
            'jmax25_values': final_jmax25,
            'jmax_values': final_jmax,
            'J_values': final_J,  # [Added] Store the full J curve
            
            # Scalar results
            'topt_aj': topt_aj_val,
            'jmax_at_topt': jmax_at_topt_val,
            'J_at_topt': J_at_topt_val,
            'aj_peak': aj_peak_val,
            'ac_at_growth': ac_at_growth_val,
            'aj_at_growth': aj_at_growth_val
        })
res_cstar = results_by_light.copy()


In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# ---------- 数据准备 ----------
data_sets = [
    {"data": full_acclimation,  "title_prefix": "Full Acclimation"}, 
    {"data": res_gamma, "title_prefix": "Fixed $\Gamma^*$"},
    {"data": res_chi,   "title_prefix": "Fixed $\chi$"},
    {"data": res_phi,   "title_prefix": "Fixed $\phi_0$"}
]

current_light = 1800  
colors9 = ["#0b0c48", "#313695", "#4575b4", "#74add1", "#abd9e9", 
           "#F9D057", "#fdae61", "#f46d43", "#a50026"]
palette_temps = np.arange(0, 41, 2.5)
temp_color_map = dict(zip(palette_temps, colors9))

import matplotlib.cm as cm

# ---------- 修改颜色方案 ----------
# 1. 自动获取数据中所有的驯化温度
all_acclim_temps = []
for ds in data_sets:
    res = ds["data"].get(current_light, [])
    all_acclim_temps.extend([rec['acclim_temp'] for rec in res])
unique_temps = sorted(list(set(all_acclim_temps)))
n_colors = len(unique_temps)

# 2. 方案选择：使用 plasma 色谱 (从深紫到粉橘)
# 这个色谱完全没有“明黄色”，且对比度极高，每一根线在白色背景上都很清晰
cmap = plt.get_cmap('viridis') 

# 3. 截取色谱：为了避免最高温处出现过亮的颜色，我们只取色谱的 0% 到 90%
colors_dynamic = cmap(np.linspace(0, 1, n_colors))

temp_color_map = dict(zip(unique_temps, colors_dynamic))
# --------------------------------
# 4. 创建映射字典
temp_color_map = dict(zip(unique_temps, colors_dynamic))
# ---------- 绘图设置 ----------
fig, axes = plt.subplots(4, 2, figsize=(16, 20), sharex=True, sharey=False)

# 调整间距，为右侧 legend 留出空间
plt.subplots_adjust(hspace=0.22, wspace=0.25, right=0.88)

plot_vars = [
    {"key": "aj_values", "label": "$A_j$",  "unit": "µmol m$^{-2}$ s$^{-1}$"},
    {"key": "gpp_values", "label": "A", "unit": "µmol m$^{-2}$ s$^{-1}$"}
]

panel_labels = ["(a)", "(b)", "(c)", "(d)", "(e)", "(f)", "(g)", "(h)"]

# ---------- 核心循环 ----------
for row, data_info in enumerate(data_sets):
    current_res = data_info["data"].get(current_light, [])
    
    for col, var_info in enumerate(plot_vars):
        ax = axes[row, col]
        
        for rec in sorted(current_res, key=lambda d: d['acclim_temp']):
            t_acc = rec['acclim_temp']
            x = np.array(rec['instant_temps'])
            y = np.array(rec.get(var_info["key"]))
            
            if y is not None and len(y) > 0:
                line_color = temp_color_map.get(t_acc, "black")
                
                # 1. 绘制原始线条
                ax.plot(x, y, color=line_color, linewidth=2.5, label=f"{t_acc}°C")
                
                # 2. 找到峰值并加点
                idx_max = np.argmax(y) # 找到最大值的索引
                peak_x = x[idx_max]
                peak_y = y[idx_max]
                
                # 绘制峰值点：s=100控制大小，edgecolors='white'增加辨识度，zorder确保点在最上方
                ax.scatter(peak_x, peak_y, color=line_color, s=100, 
                           edgecolors='white', linewidth=1.2, zorder=5)
        
        # 面板标题与装饰
        p_idx = row * 2 + col
        ax.set_title(f"{panel_labels[p_idx]} {data_info['title_prefix']} - {var_info['label']}", 
                     loc='left', fontsize=20, fontweight='bold', pad=10)
        ax.set_ylabel(f"{var_info['label']}\n({var_info['unit']})", fontsize=20)
        ax.tick_params(axis='both', direction='in', length=6, width=1.5, labelsize=20)
        ax.set_xlim(0, 55)
        ax.set_ylim(0,85)
        for spine in ['top', 'right']:
            ax.spines[spine].set_visible(False)

# 底部 X 轴标签
for ax in axes[3, :]:
    ax.set_xlabel('Instantaneous Temperature (°C)', fontsize=20)

# ---------- 全局图例 ----------
# 从第一行第一个子图获取图例句柄
handles, labels = axes[0, 0].get_legend_handles_labels()
fig.legend(handles, labels, 
           title="Growth Temp", 
           loc='center right', 
           bbox_to_anchor=(1, 0.5), 
           fontsize=18, 
           title_fontsize=20, 
           frameon=False)

plt.show()

In [ ]:
# ---------- 修改颜色方案 ----------
palette_temps = np.arange(10, 41, 2.5)
current_light = 1800  

# 1. 自动获取数据中所有的驯化温度
all_acclim_temps = []
for ds in data_sets:
    res = ds["data"].get(current_light, [])
    all_acclim_temps.extend([rec['acclim_temp'] for rec in res])
unique_temps = sorted(list(set(all_acclim_temps)))
n_colors = len(unique_temps)

# 2. 方案选择：使用 plasma 色谱 (从深紫到粉橘)
# 这个色谱完全没有“明黄色”，且对比度极高，每一根线在白色背景上都很清晰
cmap = plt.get_cmap('viridis') 

# 3. 截取色谱：为了避免最高温处出现过亮的颜色，我们只取色谱的 0% 到 90%
colors_dynamic = cmap(np.linspace(0, 1, n_colors))

temp_color_map = dict(zip(unique_temps, colors_dynamic))
# --------------------------------
# 4. 创建映射字典
temp_color_map = dict(zip(unique_temps, colors_dynamic))
# ---------- 绘图设置 ----------
fig, axes = plt.subplots(2, 2, figsize=(16, 10), sharex=True, sharey=False)

# 调整间距，为右侧 legend 留出空间
plt.subplots_adjust(hspace=0.22, wspace=0.25, right=0.88)

plot_vars = [
    {"key": "aj_values", "label": "$A_j$",  "unit": "µmol m$^{-2}$ s$^{-1}$"},
    {"key": "gpp_values", "label": "A", "unit": "µmol m$^{-2}$ s$^{-1}$"}
]

panel_labels = ["(a)", "(b)", "(c)", "(d)", "(e)", "(f)", "(g)", "(h)"]

# ---------- 核心循环 ----------
for row, data_info in enumerate(data_sets):
    current_res = data_info["data"].get(current_light, [])
    
    for col, var_info in enumerate(plot_vars):
        ax = axes[row, col]
        
        for rec in sorted(current_res, key=lambda d: d['acclim_temp']):
            t_acc = rec['acclim_temp']
            x = np.array(rec['instant_temps'])
            y = np.array(rec.get(var_info["key"]))
            
            if y is not None and len(y) > 0:
                line_color = temp_color_map.get(t_acc, "black")
                
                # 1. 绘制原始线条
                ax.plot(x, y, color=line_color, linewidth=2.5, label=f"{t_acc}°C")
                
                # 2. 找到峰值并加点
                idx_max = np.argmax(y) # 找到最大值的索引
                peak_x = x[idx_max]
                peak_y = y[idx_max]
                
                # 绘制峰值点：s=100控制大小，edgecolors='white'增加辨识度，zorder确保点在最上方
                ax.scatter(peak_x, peak_y, color=line_color, s=100, 
                           edgecolors='white', linewidth=1.2, zorder=5)
        
        # 面板标题与装饰
        p_idx = row * 2 + col
        ax.set_title(f"{panel_labels[p_idx]} {data_info['title_prefix']} - {var_info['label']}", 
                     loc='left', fontsize=20, fontweight='bold', pad=10)
        ax.set_ylabel(f"{var_info['label']}\n({var_info['unit']})", fontsize=20)
        ax.tick_params(axis='both', direction='in', length=6, width=1.5, labelsize=20)
        ax.set_xlim(0, 55)
        ax.set_ylim(0,50)
        for spine in ['top', 'right']:
            ax.spines[spine].set_visible(False)

# 底部 X 轴标签
for ax in axes[1, :]:
    ax.set_xlabel('Instantaneous Temperature (°C)', fontsize=20)

# ---------- 全局图例 ----------
# 从第一行第一个子图获取图例句柄
handles, labels = axes[0, 0].get_legend_handles_labels()
fig.legend(handles, labels, 
           title="Growth Temp", 
           loc='center right', 
           bbox_to_anchor=(1, 0.5), 
           fontsize=18, 
           title_fontsize=20, 
           frameon=False)

plt.show()